In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model='gpt-4o')

In [4]:
import warnings
from langchain import hub

warnings.filterwarnings("ignore", category=UserWarning, module='langchain')
load_dotenv()

prompt = hub.pull('rlm/rag-prompt')

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [8]:
response = llm.invoke('안녕')

In [9]:
response

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc07abaa-4aea-48db-a7e7-f6866bf5ebaf-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19})

In [10]:
response.content

'안녕하세요! 어떻게 도와드릴까요?'

In [23]:
context = '''## 영속성 관리\n\nJPA를 이해하기 위해서는 `영속성 컨텍스트`를 이해해야 한다.\n\n영속성 컨텍스트를 이해해야 JPA의 내부동작을 이해할 수 있다.\n\n## 엔티티 매니저 팩토리와 엔티티 매니저\n\n`EntityManagerFactory`는 고객의 요청이 올 때마다 `EntityManager`를 만들고, `EntityManager`는 커넥션을 이용해 DB를 사용하게 된다.\n\n## 영속성 컨텍스트 (PersistenceContext)\n\n- JPA를 이해하는데 가장 중요한 용어\n- **엔티티를 영구 저장하는 환경**이라는 뜻\n- EntityManager.persist(entity);\n    - **사실 DB에 저장한다는게 아니라 Entity를 영속성 컨텍스트에 저장한다는 의미이다**\n- 영속성 컨텍스트는 논리적인 개념으로, 눈에 보이지 않는다\n    - 엔티티 매니저를 통해서 영속성 컨텍스트에 접근하게 된다\n    - 엔티티 매니저가 생성되면 1:1로 영속성 컨텍스트가 생성되게 된다\n\n## 엔티티의 생명주기\n\n엔티티는 생명주기를 가진다.\n\n- **비영속 (new/transient)**\n    - `new`를 이용해 새롭게 생성한 객체\n    - 영속성 컨텍스트와 전혀 관계가 없는 새로운 상태\n- **영속 (managed)**\n    - 영속성 컨텍스트에 관리되는 상태\n- **준영속 (detached)**\n    - 영속성 컨텍스트에 저장되었다가 분리된 상태\n- **삭제 (removed)**\n    - 삭제된 상태\n\n### 비영속\n\n```java\nMember member = new Member();\nmember.setId(\"member1\");\nmember.setUsername(\"회원1\");\n```\n\n객체를 생성하고 영속성 컨텍스트와는 아무 관계가 없는 상태이다.\n\n### 영속\n\n```java\nEntityManager em = emf.createEntityManager();\nem.getTransaction().begin();\n\nem.persist(member);\n```\n\n`entitymanager`에 `member`를 넣으면 영속성 컨텍스트에 member가 들어가고, `영속 상태`가 된다.\n\n영속상태가 되더라도 바로 쿼리가 날라가는 것이 아니다. 트랜잭션을 커밋하는 순간에 영속성 컨텍스트가 쿼리로 날아가게 되는 것이다.\n\n뒤에서 더 자세히 설명한다.\n\n### 준영속, 삭제\n\n준영속 : 영속 컨텍스트에 있던 요소를 영속 컨텍스트에서 지우겠다는 의미이다\n\n삭제 : DB 삭제를 요청하는 상태이다\n\n```java\nem.detach(member); // 회원 엔티티를 영속성 컨텍스트에서 분리, 준영속 상태\nem.remove(member); // 객체를 삭제한 상태 (삭제)\n```\n\n## 영속성 컨텍스트의 이점\n\n애플리케이션과 DB 사이에 중간 계층을 두는 것이다\n\n- 1차 캐시\n- 동일성(identity) 보장\n- 트랜잭션을 지원하는 쓰기 지연 (Transactional write-behind)\n- 변경 감지 (Dirty Checking)\n- 지연 로딩 (Lazy Loading)\n
'''

In [62]:
result = llm.invoke(f'''
# 요구 사항
- **html문서로 작성해주세요. 가독성을 고려해야 합니다.**
- **문장이 길다고 판단되면 다음 줄에 작성해주세요.**
- 내용: []에 오는 내용을 사람이 읽기 좋게 가독성을 고려해서 요약해주세요.
- 모르는 정보면 모른다고 답변해주세요.
- 답변 전 내용을 스스로 검증하고 대답해주세요.
- url을 제공해주는 경우 해당 url이 정상적인 http 상태를 반환하는 경우에만 제공해주세요
- 종결 어미는 '~이다'로 끝내주세요.
- '\n'보다는 태그를 이용한 줄바꿈을 이용해주세요 
- 코드가 포함된다면 ``` ``` 안에 작성해주세요

## 강의 목적
- 강의 목적 앞에는 "강의 목표: " 라는 말을 붙여주세요

## 용어 설명
- 강의에서 정의를 설명하는 부분을 정리해주세요
- {{용어 이름}} : {{용어 설명}} 과 같이 정리해주세요
- 용어 설명에는 절대 절대 종결 어미를 붙이지 마세요

## 본문 요약
- 본문 요약 내용은 주제에 맞게 2개 이상으로 나누어 작성해주세요
- 강의 요약 내용은 p 태그로 보여주고 display: block style을 주어서 문장이 너무 길면 다음 줄에 나오도록 작성해주세요.
- 본문의 내용은 소주제에 따라 각각의 문단으로 정리해주세요.
- 각각의 본문 요약에 소제목을 작성해주세요
- 코드가 포함된다면 코드는 반드시 포함시켜주세요.
  - 코드는 ``` ``` 태그로 감싸서 작성해주세요
  - 기존 코드가 있고 이를 최적화할 수 있는 코드를 작성할 수 있다면 최적화된 코드를 함께 작성해주세요

## 강의 요약
- 10문장 이하로 작성해주세요
- 결론에는 강의의 가장 중요한 내용만 넣어주세요
- 개조식으로 작성해주세요
- ul 태그로 묶고 각 문장을 li 태그로 작성해주세요

### 관련 자료 추천
- [본문 요약] 위치에는 강의 내용을 정리한 글을 작성하고 이해를 돕는 내용을 추가해주세요.  
- 관련 레퍼런스에는 해당 강의 주제와 관련해서 더 공부하기 좋은 내용을 작성해주세요.
  - 책을 추천한다면 'https://search.kyobobook.co.kr/search?keyword={{책이름}}'을 답변해주세요
  - 공식 문서를 추천한다면 공식 문서의 url을 포함해서 답변해주세요
  - 관련 웹 사이트나 웹 문서를 추천한다면 해당 문서의 url을 포함해서 답변해주세요


# 답변 양식 예시
<html>
제목 : 
용어 정리
- 
- 
- 

본문요약
  본문요약1
  ~~ 본문 내용~~
  ```java
    Member member = new Member();
    member.setId("member1");
    member.setUsername("회원1");
  ```
  
  최적화된 코드
  ```java
    // 최적화되었거나 추천하는 코드를 작성
    // 추천하는 코드가 없다면 작성하지 않아도 됩니다
  ```  
    
    
  본문요약2
  ~~ 본문 내용~~

결론
- 
- 
- 

관련 레퍼런스
</html>
---
내용: [{context}]
''')

In [63]:
result.content

'```html\n<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <title>JPA 영속성 관리</title>\n    <style>\n        p {\n            display: block;\n            margin-bottom: 10px;\n        }\n    </style>\n</head>\n<body>\n    <h1>JPA 영속성 관리</h1>\n    \n    <h2>강의 목표</h2>\n    <p>JPA의 영속성 컨텍스트 이해와 이를 통한 JPA 내부 동작 이해</p>\n    \n    <h2>용어 설명</h2>\n    <ul>\n        <li>영속성 컨텍스트 : 엔티티를 영구 저장하는 환경</li>\n        <li>엔티티 매니저 팩토리 : 고객의 요청이 올 때마다 엔티티 매니저를 만드는 팩토리 클래스</li>\n        <li>엔티티 매니저 : 커넥션을 이용해 DB를 사용하고 영속성 컨텍스트를 관리하는 클래스</li>\n        <li>비영속 : 영속성 컨텍스트와 전혀 관계가 없는 새로운 객체 상태</li>\n        <li>영속 : 영속성 컨텍스트에 의해 관리되는 엔티티 상태</li>\n        <li>준영속 : 영속성 컨텍스트에서 분리된 엔티티 상태</li>\n        <li>삭제 : DB 삭제를 요청하는 엔티티 상태</li>\n    </ul>\n    \n    <h2>본문 요약</h2>\n    \n    <h3>영속성 컨텍스트</h3>\n    <p>\n        영속성 컨텍스트는 \n        JPA를 이해하는 데 가장 중요한 개념으로, 엔티티를 영구 저장하는 환경이다. \n        엔티티 매니저를 통해 접근하며, 엔티티 매니저가 생성되면 1:1로 영속성 컨텍스트가 생성된다. \n        이 컨텍스트는 논리적인 개념으로 눈에 보이지 않으며, \n  

In [41]:
res = llm.invoke(f"""
{result.content}의 내용을 html로 작성해주세요
""")

In [42]:
res.content

'```html\n<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>JPA의 영속성 관리 및 엔티티 생명주기 강의</title>\n</head>\n<body>\n    <header>\n        <h1>JPA의 영속성 관리 및 엔티티 생명주기</h1>\n    </header>\n    <main>\n        <section>\n            <h2>강의 목적</h2>\n            <p>강의 목표: JPA의 영속성 컨텍스트와 엔티티 생명주기에 대한 이해를 돕는 것이다</p>\n        </section>\n\n        <section>\n            <h2>용어 설명</h2>\n            <ul>\n                <li><strong>영속성 컨텍스트:</strong> 엔티티를 영구 저장하는 환경을 의미</li>\n                <li><strong>EntityManagerFactory:</strong> 고객의 요청 시 EntityManager를 생성하는 팩토리</li>\n                <li><strong>EntityManager:</strong> DB와의 연결을 관리하며 영속성 컨텍스트에 접근하는 관리자</li>\n            </ul>\n        </section>\n\n        <section>\n            <h2>소주제</h2>\n\n            <article>\n                <h3>1. 엔티티 매니저 팩토리와 엔티티 매니저</h3>\n                <p>EntityManagerFactory는 고객의 요청이 있을 때마다 EntityManager를 생성하는